In [2]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')

In [3]:
spark = (
    SparkSession
    .builder
    .appName("Delta Lake")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .config("spark.driver.memory", "16G")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-297f4710-70ef-43cc-a5df-07b1bdccd5bd;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 in central
	found org.apache.spark#spark-token

In [4]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [5]:
from datetime import datetime
from itertools import count
from typing import List
from xmlrpc.client import Boolean
from dateutil import parser
import json
import pyspark.sql.functions as f
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, BinaryType, IntegerType, TimestampType
from pyspark.sql import Column, DataFrame, SparkSession
from loguru import logger
from delta.tables import DeltaTable
import os
from google.cloud import storage
import decimal
import base64
from io import BytesIO
from zipfile import ZipFile

In [6]:

def generete_symlink_manifest(spark:SparkSession, bucket:str) -> None:
        """
        Gera o arquivo symlink manifest para a tabela Delta do bucket de referencia
        """
        delta_table = DeltaTable.forPath(spark, bucket)
        delta_table.generate("symlink_format_manifest")


def path_exists(path, spark:SparkSession):
        """
        Dado um caminho verifica se existe ou não
        """
        sc = spark.sparkContext
        fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
            sc._jvm.java.net.URI.create("gs://" + path.split("/")[2]),
            sc._jsc.hadoopConfiguration(),
        )
        return fs.exists(sc._jvm.org.apache.hadoop.fs.Path(path))


def last_date(delta_path:str, spark:SparkSession):
        """
        Retorna a data (timestamp_kafka) do último registro da tabela Delta
        """
        if not path_exists(delta_path + 'job_control/last_date', spark):
            try:
                df_temp = (
                    spark
                    .read
                    .format("delta")
                    .load(delta_path)
                )

                list_result = (
                    df_temp
                    .agg({"timestamp_kafka": "max"}).collect()
                )

                last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
                snapshot_flag = False
            except:
                snapshot_flag = True
                last_date = ('01-01-2021 12:00AM')

            df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
            df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')
        else:
            snapshot_flag = False
            df = spark.read.format("csv").load(delta_path + 'job_control/last_date', header = 'true')
            last_date = df.collect()[0].asDict()['last_date']

        logger.info('Last date: ' + last_date)

        return parser.parse(last_date),snapshot_flag


def update_last_date(delta_path:str, df:DataFrame, spark:SparkSession):
        """
        Atualiza a data (timestamp_kafka) do último registro da tabela Delta
        """
        list_result = (
            df
            .agg({"timestamp_kafka": "max"}).collect()
        )

        last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
        df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
        df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')

        return last_date

def get_operations_counts(df:DataFrame, spark:SparkSession):
        """
        Retorna o número de registros de cada operação kafka (__op)
        """
        op_count_list =(
            df
            .groupBy('__op')
            .count()
            .collect()
        )
        
        new_data_count = 0
        updating_data_count = 0
        deleting_data_count = 0

        if len(op_count_list) > 0:
            for i in op_count_list:
                dic_temp = i.asDict()
                if (dic_temp['__op'] == 'r'):
                    new_data_count = new_data_count + dic_temp['count']
                elif (dic_temp['__op'] != 'r'):
                    updating_data_count = updating_data_count + dic_temp['count']
                elif (dic_temp['__op'] == 'd'):
                    deleting_data_count = deleting_data_count + dic_temp['count']

        new_total_data_count = new_data_count + updating_data_count + deleting_data_count
        
        return new_total_data_count, new_data_count, updating_data_count, deleting_data_count


def get_data(spark:SparkSession, bucket:str, date_colunm_name:str, date_begin:datetime, source_format:str='delta', is_landing:bool=False, is_sink:bool=False):
        """
        Carrega Dataframe filtrando os dados do bucket de referência conforme 
        data de início enviada no parametro

        :param str date_begin: Data de início para filtro dos dados
        """
        if is_landing:

            if is_sink:
                df = (
                    spark
                    .read
                    .format(source_format)
                    .load(bucket)
                    .withColumn(date_colunm_name, (f.col(date_colunm_name)/1000).cast("timestamp"))
                    .where(
                        (f.col("year")>date_begin.year)|
                        (
                            (f.col("year")==date_begin.year)&
                            (
                            (f.col("month")>date_begin.month)|
                            (
                                (f.col("month")==date_begin.month)&
                                (
                                (f.col("day")>date_begin.day)|
                                (
                                    (f.col("day")==date_begin.day)&
                                    (f.col("hour")>=date_begin.hour)&
                                    (f.col(date_colunm_name)>date_begin)
                                )
                                )
                            )
                            )          
                        ) 
                    )
                )
            else:
                schema = StructType() \
                        .add("key",BinaryType(),True) \
                        .add("value",BinaryType(),True) \
                        .add("topic",StringType(),True) \
                        .add("partition",IntegerType(),True) \
                        .add("offset",StringType(),True) \
                        .add("timestamp",TimestampType(),True)
                df = (
                    spark
                    .read
                    .format(source_format)
                    .schema(schema)
                    .load(bucket)
                    .where(
                        (f.col("year")>date_begin.year)|
                        (
                            (f.col("year")==date_begin.year)&
                            (
                            (f.col("month")>date_begin.month)|
                            (
                                (f.col("month")==date_begin.month)&
                                (
                                (f.col("day")>date_begin.day)|
                                (
                                    (f.col("day")==date_begin.day)&
                                    (f.col("hour")>=date_begin.hour)&
                                    (f.col(date_colunm_name)>date_begin)
                                )
                                )
                            )
                            )          
                        ) 
                    )
                )
        else:
            df = (
                spark
                .read
                .format(source_format)
                .load(bucket)
                .where(
                    (f.col("year")>date_begin.year)|
                    (
                        (f.col("year")==date_begin.year)&
                        (
                        (f.col("month")>date_begin.month)|
                        (
                            (f.col("month")==date_begin.month)&
                            (
                            (f.col("day")>date_begin.day)|
                            (
                                (f.col("day")==date_begin.day)&
                                (f.col("hour")>=date_begin.hour)&
                                (f.col(date_colunm_name)>date_begin)
                            )
                            )
                        )
                        )          
                    ) 
                )
            )
        return df



def write_new_data(spark:SparkSession, df:DataFrame, bucket:str, destination_format:str='delta', partition_column:str=None):
        """
        Escreve no bucket de destino os dados novos, cuja a operação kafka é 'r'
        """
        if partition_column is not None:
            df_r = df.where((f.col('__op') == 'r'))
            (
                df_r
                .write
                .format(destination_format)
                .mode("append")
                .option("checkpointLocation", bucket + "checkpoint")
                .save(bucket)
            )
        else:
            (
                df
                .write
                .partitionBy(partition_column)
                .format(destination_format)
                .mode("append")
                .option("checkpointLocation", bucket + "checkpoint")
                .save(bucket)
            )
        
        return True


def update_new_data(spark:SparkSession, df:DataFrame, bucket:str, pk:list=['pk_single']):
        """
        Escreve no bucket de destino os dados novos, cuja a operação kafka é 'c' e
        atualiza os dados existentes, cuja a operação kafka é 'u' ou 'd'
        """
        table = DeltaTable.forPath(spark, bucket)
        df_table_u = df.where((f.col('__op') != 'r'))
        df_table_u_unique = df_table_u.orderBy('timestamp_kafka', ascending = False).dropDuplicates(pk)
        merge_string = 'destination.' + pk[0] + ' = updates.' + pk[0]
        (
            table.alias('destination')
            .merge(df_table_u_unique.alias('updates'), merge_string)
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
            .execute()
        )
        
        return True


def delete_new_data(spark:SparkSession, bucket:str):
        """
        Deleta da tabela delta, no bucket de destino, os dados cuja a operação kafka é 'd'
        """
        table = DeltaTable.forPath(spark, bucket)
        table.delete(f.col('__op') == 'd')
        
        return True


def flatten_df(nested_df):
        """
        Retorna um dataframe com as colunas de um dataframe aninhado
        """
        flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
        nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

        flat_df = nested_df.select(flat_cols +
                                [f.col(nc+'.'+c)
                                    for nc in nested_cols
                                    for c in nested_df.select(nc+'.*').columns])
        return flat_df


def write_bronze_table(df:DataFrame, bucket:str):
        """
        Escreve no bucket de destino a tabela delta bronze
        """
        (
            df
            .write
            .partitionBy("year", "month", "day", "hour")
            .format("delta")
            .mode("append")
            .option("checkpointLocation", bucket + "checkpoint")
            .save(bucket)
        )

        return True

def get_schema(df:DataFrame):
    json_str = df.limit(1).toPandas()['fixedValue'][0]
    logger.info(json_str)
    json_fields = json.loads(json_str)['schema']['fields']
    struct_fields = []
    for field in json_fields:
        struct_fields.append(StructField(field['field'], StringType()))
        
    return StructType(struct_fields)

def decode_data_frame(df:DataFrame, schema:StructType):
    df_decoded = ( 
        df
        .withColumn('fixedValue', f.col("value").cast("string"))
        .select(f.get_json_object('fixedValue',"$.payload").alias('payload'), 'timestamp_kafka')
        .withColumnRenamed('timestamp', 'timestamp_kafka')
        .select(f.from_json(f.col('payload'), schema).alias("DF"), 'timestamp_kafka')
        .select("DF.*" , 'timestamp_kafka')
    )
    
    return df_decoded

def upload_blob_from_memory(bucket_name, contents, destination_blob_name):
    """Uploads a file to the bucket."""

    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The contents to upload to the file
    # contents = "these are my contents"

    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_string(contents)

def zip_file(nome_arquivo:str,contents):
    file_zip = BytesIO()
    with ZipFile(file_zip, 'w') as zipObj2:
        zipObj2.writestr(nome_arquivo,contents)
        zipObj2.close()
    return file_zip.getvalue()

def send_to_ftp(df:DataFrame, brand:str, table_name:str):

    temp_path = '/hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/temp/' + brand + '/'

    file_name = brand + '_' + table_name + '_' + datetime.today().strftime("%Y%m%d_%H%M%S")

    logger.info('Creating file...')
    df.coalesce(1).write.option("header",True).option("delimiter","|").csv('gs:/' + temp_path + file_name)

    storage_client = storage.Client()
    bucket = storage_client.get_bucket('hering-datalake-prod-southamerica-east1-bucket-gold-zone')

    blobs_list = list(bucket.list_blobs(prefix='oto/temp/' + brand + '/' + file_name + '/'))

    for file in blobs_list:
        try:
            if file.name.split('.')[-1] == 'csv':
                blob = file
        except:
            pass

    logger.info('Renaming file...')
    bucket.rename_blob(blob, 'oto/temp/' + brand + '/' + file_name + '.csv')

    nome_arquivo_csv = file_name + '.csv'
    nome_arquivo_zip = file_name + '.zip'
    bucket_name = 'hering-datalake-prod-southamerica-east1-bucket-gold-zone'
    blob = bucket.blob('oto/temp/' + brand + '/' + nome_arquivo_csv)
    contents = blob.download_as_string()

    contents_zip = zip_file(nome_arquivo_csv,contents)
    upload_blob_from_memory(bucket_name,contents_zip,'oto/temp/' + brand + '/' + nome_arquivo_zip)



        
    return None

@f.udf(returnType=DoubleType())
def big_java_decimal_to_double_python(big_decimal):
    bytes_val = base64.decodebytes(big_decimal.encode())
    bval = "".join("{0:08b}".format(c) for c in bytes_val)
    intval = int(bval, 2)

    return intval/(10**2)


def write_clientes_abordaveis(spark:SparkSession, df:DataFrame, bucket:str, brand:str):

    clientes_abordaveis_path = '/hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/clientes_abordaveis/' + brand + '/'
    file_name = brand + '_' + 'clientes_grupocontrole' + '_' + datetime.today().strftime("%Y%m%d_%H%M%S")

    logger.info('Creating file...')
    df.coalesce(1).write.option("header",True).option("delimiter","|").csv('gs:/' + clientes_abordaveis_path + file_name)

    storage_client = storage.Client()
    bucket = storage_client.get_bucket('hering-datalake-prod-southamerica-east1-bucket-gold-zone')

    blobs_list = list(bucket.list_blobs(prefix='oto/clientes_abordaveis/' + brand + '/' + file_name + '/'))

    for file in blobs_list:
        try:
            if file.name.split('.')[-1] == 'csv':
                blob = file
        except:
            pass

    logger.info('Renaming file...')
    bucket.rename_blob(blob, 'oto/clientes_abordaveis/' + brand + '/' + file_name + '.csv')

    nome_arquivo_csv = file_name + '.csv'
    nome_arquivo_zip = file_name + '.zip'
    bucket_name = 'hering-datalake-prod-southamerica-east1-bucket-gold-zone'
    blob = bucket.blob('oto/clientes_abordaveis/' + brand + '/' + nome_arquivo_csv)
    contents = blob.download_as_string()

    contents_zip = zip_file(nome_arquivo_csv,contents)
    upload_blob_from_memory(bucket_name,contents_zip,'oto/clientes_abordaveis/' + brand + '/' + nome_arquivo_zip)

In [7]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType
from dateutil import parser

In [11]:
LANDING_PATH_ANTIGO = 'gs://hering-datalake-prod-southamerica-east1-bucket-sandbox-zone/3bnsvher327/coml/coml/eih.bnsvher327.coml.COML.ARARTIGO/'
LANDING_PATH_NOVO = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher327/coml/coml/eih.bnsvher327.coml.COML.ARARTIGO'


In [12]:
last_date = parser.parse('21-03-1800 08:00AM')
date_begin = last_date

In [13]:
antigo_sink = get_data(spark, LANDING_PATH_ANTIGO, 'ts_ms', last_date, 'json', True, True)

AnalysisException: Path does not exist: gs://hering-datalake-prod-southamerica-east1-bucket-sandbox-zone/3bnsvher327/coml/coml/eih.bnsvher327.coml.COML.ARARTIGO

In [82]:
novo_sink = get_data(spark, LANDING_PATH_NOVO, 'ts_ms', last_date, 'json', True, True)

In [86]:
cols = ('before', 'source')
df_table = novo_sink.drop(*cols)
df_table = df_table.withColumnRenamed("ts_ms", "timestamp_kafka").withColumnRenamed("op", "__op")

In [88]:
df_table_flat = flatten_df(df_table)

In [90]:
df_table_flat.limit(60000).toPandas()

__op transaction         timestamp_kafka  year  month  day  hour  \
0        c        None 2022-07-27 12:46:09.606  2022      8    5    15   
1        u        None 2022-07-27 12:46:09.606  2022      8    5    15   
2        u        None 2022-07-27 12:47:11.192  2022      8    5    15   
3        u        None 2022-07-27 12:48:19.380  2022      8    5    15   
4        u        None 2022-07-27 12:48:21.189  2022      8    5    15   
...    ...         ...                     ...   ...    ...  ...   ...   
59995    r        None 2022-07-15 11:46:42.149  2022      7   15     8   
59996    r        None 2022-07-15 11:46:42.149  2022      7   15     8   
59997    r        None 2022-07-15 11:46:42.149  2022      7   15     8   
59998    r        None 2022-07-15 11:46:42.149  2022      7   15     8   
59999    r        None 2022-07-15 11:46:42.149  2022      7   15     8   

       ARARTDET_ID  ARARTIGO_ID  ARCANDIS_ID  ...  INRESEXP  INSEMCLA  \
0              NaN       485799          1.0  ...         N         N   
1              NaN       485799          1.0  ...         N         N   
2              NaN       457928          1.0  ...         N         N   
3              NaN       472412          1.0  ...         N         N   
4              NaN       475906          1.0  ...         N         N   
...            ...          ...          ...  ...       ...       ...   
59995          NaN        90538          NaN  ...         N         N   
59996          NaN        90542          NaN  ...         N         N   
59997          NaN        90544          NaN  ...         N         N   
59998          NaN        90546          NaN  ...         N         N   
59999          NaN        90555          NaN  ...         N         N   

       INSITUAC  INTIPCOD  INTIPDIV  INTIPTAB  QTMULEMB  QTMULVEN  QTUNIART  \
0             A         M         T         T         1         1         1   
1             A         M         T         T         1         1         1   
2             E         M         P         T         1         1         1   
3             E         M         P         T         1         1         1   
4             E         M         T         T         1         1         1   
...         ...       ...       ...       ...       ...       ...       ...   
59995         C         M         P         T         1         1         1   
59996         C         M         P         T         1         1         1   
59997         C         M         P         T         1         1         1   
59998         C         M         P         T         1         1         1   
59999         C         M         P         T         1         1         1   

       VLCTOPAD  
0         72.70  
1         72.70  
2          0.00  
3          0.00  
4          0.00  
...         ...  
59995      None  
59996      None  
59997      None  
59998      None  
59999      None  

[60000 rows x 99 columns]

In [83]:
novo_sink.count()

589387

In [71]:
df = (
        spark
        .read
        .format('json')
        .load(LANDING_PATH_ANTIGO)

)

In [72]:
df.count()

53958

In [73]:
cols = ('before', 'source')
df_table = df.drop(*cols)
df_table = df_table.withColumnRenamed("ts_ms", "timestamp_kafka").withColumnRenamed("op", "__op")


In [74]:
df_table_flat = flatten_df(df_table)

In [75]:
df_table_flat.toPandas()

__op transaction  timestamp_kafka  year  month  day  hour  ARARTDET_ID  \
0        u        None    1658490414300  2022      8    5    15          NaN   
1        u        None    1658490414333  2022      8    5    15          NaN   
2        u        None    1658490414335  2022      8    5    15          NaN   
3        u        None    1658490414346  2022      8    5    15          NaN   
4        u        None    1658490414348  2022      8    5    15          NaN   
...    ...         ...              ...   ...    ...  ...   ...          ...   
53953    u        None    1659712525432  2022      8    5    15          NaN   
53954    u        None    1659712399311  2022      8    5    15          NaN   
53955    u        None    1659712406440  2022      8    5    15          NaN   
53956    u        None    1659712574457  2022      8    5    15          NaN   
53957    u        None    1659712424286  2022      8    5    15          NaN   

       ARARTIGO_ID  ARCANDIS_ID  ...  INRESEXP INSEMCLA  INSITUAC  INTIPCOD  \
0           427819          1.0  ...         N        N         E         M   
1           457647          1.0  ...         N        N         E         M   
2           471332          1.0  ...         N        N         E         M   
3           467541          1.0  ...         N        N         E         M   
4           457645          1.0  ...         N        N         E         M   
...            ...          ...  ...       ...      ...       ...       ...   
53953       405339          1.0  ...         N        N         E         M   
53954       309886          1.0  ...         N        N         E         M   
53955       337087          1.0  ...         N        N         E         M   
53956       405341          1.0  ...         N        N         E         M   
53957       369764          1.0  ...         N        N         E         M   

       INTIPDIV INTIPTAB QTMULEMB  QTMULVEN  QTUNIART  VLCTOPAD  
0             P        T        1         1         1     22.48  
1             P        T        1         1         1      0.00  
2             P        T        1         1         1      0.00  
3             P        T        1         1         1      0.00  
4             P        T        1         1         1      0.00  
...         ...      ...      ...       ...       ...       ...  
53953         P        T        1         1         1      6.29  
53954         P        T        1         1         1      7.59  
53955         P        T        1         1         1      4.69  
53956         P        T        1         1         1      6.29  
53957         P        T        1         1         1     15.38  

[53958 rows x 99 columns]

In [61]:
df = (
        spark
        .read
        .format('json')
        .load(LANDING_PATH_NOVO)
        .withColumn('ts_ms', (f.col('ts_ms')/1000).cast("timestamp"))
        .where(
            (f.col("year")>date_begin.year)|
            (
                (f.col("year")==date_begin.year)&
                (
                (f.col("month")>date_begin.month)|
                (
                    (f.col("month")==date_begin.month)&
                    (
                    (f.col("day")>date_begin.day)|
                    (
                        (f.col("day")==date_begin.day)&
                        (f.col("hour")>=date_begin.hour)&
                        (f.col('ts_ms')>date_begin)
                    )
                    )
                )
                )          
            )
        )
)

In [62]:
df.limit(10).toPandas()

after before op  \
0  (None, 71163, None, None, None, None, 1, 77, N...   None  r   
1  (None, 71168, None, None, None, None, 1, 77, N...   None  r   
2  (None, 71170, None, None, None, None, 1, 77, N...   None  r   
3  (None, 71171, None, None, None, None, 1, 77, N...   None  r   
4  (None, 71173, None, None, None, None, 1, 77, N...   None  r   
5  (None, 71174, None, None, None, None, 1, 77, N...   None  r   
6  (None, 71176, None, None, None, None, 1, 77, N...   None  r   
7  (None, 71179, None, None, None, None, 1, 77, N...   None  r   
8  (None, 71180, None, None, None, None, 1, 77, N...   None  r   
9  (None, 71183, None, None, None, None, 1, 77, N...   None  r   

                                              source transaction  \
0  (None, oracle, CORP1, None, bnsvher327.coml, C...        None   
1  (None, oracle, CORP1, None, bnsvher327.coml, C...        None   
2  (None, oracle, CORP1, None, bnsvher327.coml, C...        None   
3  (None, oracle, CORP1, None, bnsvher327.coml, C...        None   
4  (None, oracle, CORP1, None, bnsvher327.coml, C...        None   
5  (None, oracle, CORP1, None, bnsvher327.coml, C...        None   
6  (None, oracle, CORP1, None, bnsvher327.coml, C...        None   
7  (None, oracle, CORP1, None, bnsvher327.coml, C...        None   
8  (None, oracle, CORP1, None, bnsvher327.coml, C...        None   
9  (None, oracle, CORP1, None, bnsvher327.coml, C...        None   

                    ts_ms  year  month  day  hour  
0 2022-07-15 11:46:41.927  2022      7   15     8  
1 2022-07-15 11:46:41.927  2022      7   15     8  
2 2022-07-15 11:46:41.927  2022      7   15     8  
3 2022-07-15 11:46:41.927  2022      7   15     8  
4 2022-07-15 11:46:41.927  2022      7   15     8  
5 2022-07-15 11:46:41.928  2022      7   15     8  
6 2022-07-15 11:46:41.928  2022      7   15     8  
7 2022-07-15 11:46:41.928  2022      7   15     8  
8 2022-07-15 11:46:41.928  2022      7   15     8  
9 2022-07-15 11:46:41.928  2022      7   15     8

In [27]:
df.count()

535204

In [28]:
cols = ('before', 'source')
df = df.drop(*cols)
df = df.withColumnRenamed("ts_ms", "timestamp_kafka").withColumnRenamed("op", "__op")

In [29]:
df_table_flat = flatten_df(df)

In [30]:
df_table_flat.limit(10).toPandas()

__op transaction         timestamp_kafka  year  month  day  hour  \
0    r        None 2022-07-15 11:46:41.927  2022      7   15     8   
1    r        None 2022-07-15 11:46:41.927  2022      7   15     8   
2    r        None 2022-07-15 11:46:41.927  2022      7   15     8   
3    r        None 2022-07-15 11:46:41.927  2022      7   15     8   
4    r        None 2022-07-15 11:46:41.927  2022      7   15     8   
5    r        None 2022-07-15 11:46:41.928  2022      7   15     8   
6    r        None 2022-07-15 11:46:41.928  2022      7   15     8   
7    r        None 2022-07-15 11:46:41.928  2022      7   15     8   
8    r        None 2022-07-15 11:46:41.928  2022      7   15     8   
9    r        None 2022-07-15 11:46:41.928  2022      7   15     8   

   ARARTDET_ID  ARARTIGO_ID  ARCANDIS_ID  ...  INRESEXP  INSEMCLA  INSITUAC  \
0          NaN        71163          NaN  ...         N         S         C   
1          NaN        71168          NaN  ...         N         S         C   
2          NaN        71170          NaN  ...         N         S         C   
3          NaN        71171          NaN  ...         N         S         C   
4          NaN        71173          NaN  ...         N         S         C   
5          NaN        71174          NaN  ...         N         S         C   
6          NaN        71176          NaN  ...         N         S         C   
7          NaN        71179          NaN  ...         N         S         C   
8          NaN        71180          NaN  ...         N         S         C   
9          NaN        71183          NaN  ...         N         S         C   

   INTIPCOD  INTIPDIV  INTIPTAB  QTMULEMB  QTMULVEN  QTUNIART  VLCTOPAD  
0         M         P         T         1         1         1      None  
1         M         P         T         1         1         1      None  
2         M         P         T         1         1         1      None  
3         M         P         T         1         1         1      None  
4         M         P         T         1         1         1      None  
5         M         P         T         1         1         1      None  
6         M         P         T         1         1         1      None  
7         M         P         T         1         1         1      None  
8         M         P         T         1         1         1      None  
9         M         P         T         1         1         1      None  

[10 rows x 99 columns]

In [ ]:
data_json

In [ ]:
# Reading last bronze data
try:
    df_bronze = (
        spark
        .read
        .format("delta")
        .load(bronze_path)
    )

    list_result = (
        df_bronze
        .agg({"timestamp_kafka": "max"}).collect()
    )
    
    last_date = list_result[0].asDict()['max(timestamp_kafka)']
    
except:
    last_date = parser.parse("01-01-2021 12:00AM")

In [ ]:
print(last_date)

In [ ]:
# Reading landing data
df_table = (
    spark
    .read
    .format("avro")
    .load(landing_path)
    .where((f.col("year")>=last_date.year)&(f.col("month")>=last_date.month)&(f.col("day")>=last_date.day))
    .where(f.col("timestamp")>last_date)
    .withColumnRenamed("timestamp", "timestamp_kafka")
)

In [ ]:
df_table.toPandas()

In [ ]:
from_avro_options= {"mode":"PERMISSIVE"}
df_temp = (
    df_table
    .withColumn('fixedValue', f.expr("substring(value, 6, length(value)-5)"))
    .select(from_avro("fixedValue", data_json['schema'], from_avro_options), 'offset', 'timestamp_kafka',"year", "month", "day", "hour")
)

In [ ]:
df_table_flat = flatten_df(df_temp)

In [ ]:
(
    df_table_flat
    .write
    .partitionBy("year", "month", "day", "hour")
    .format("delta")
    .mode("append")
    .option("checkpointLocation", bronze_path + "checkpoint")
    .save(bronze_path)
)

In [ ]:
##################################
### Verificar está funcionando ###
##################################

In [ ]:
count_bronze = (
    spark
    .read
    .format("delta")
    .load(bronze_path)
    .count()
)

print (count_bronze)

In [ ]:
landing_path_snapshot = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/ecommerce/bnsvher602/linx_db/dbo/produtos_snapshot/'

count_landing = (
    spark
    .read
    .format("avro")
    .load(landing_path)
    .count()
)

count_landing_snap = (
    spark
    .read
    .format("avro")
    .load(landing_path_snapshot)
    .count()
)


print (count_landing + count_landing_snap)